In [2]:
import numpy as np
from matplotlib import pyplot as plt
from dftpy.ions import Ions
from ase.build import bulk
from ase.io import read
from ase.lattice.spacegroup import Spacegroup
from ase.lattice.spacegroup import crystal
import ase.io

from scipy.optimize import minimize
from dscribe.descriptors import CoulombMatrix, SineMatrix, EwaldSumMatrix, SOAP

/opt/homebrew/lib/python3.11/site-packages/ase/lattice/spacegroup.py:5: UserWarning: Moved to ase.spacegroup
  warnings.warn('Moved to ase.spacegroup')


In [1]:
pwd

'/Users/valeria/Documents/aiWT/Final_version/wt/wt/organize_data'

In [3]:
import os
os.chdir('../..')
path_file = os.getcwd()

#### Code 
Since we observed a pattern in $\rho_{0}$ we can learn it with the data we generated. To relate $\rho_{0}$ with the phases of Si and its size we built Ewal sum matrix descriptors for all the phases. 

#### Creating the database 
We create the Ewal sum matrix for the five phases of silicon.

In [46]:
from dscribe.descriptors import CoulombMatrix, SineMatrix, EwaldSumMatrix, SOAP
traj_Btin = []
tral_fcc = []
traj_bcc = []
traj_8cd = []

traj_bct5 = []
Ewal_matrices = []
Sine_matrices = []
Sine_gradient = []
Soap_matrices = []

r = np.linspace(0.8, 1.4, 30)
for i, d in enumerate(r):
    si_Btin = Ions.from_ase(crystal('Si',[(0,0,0)],spacegroup=141,cellpar = [4.81,4.81,2.65, 90,90,90]))
    cell = si_Btin.get_cell()
    si_Btin.set_cell(cell * d, scale_atoms=True)
    
    si_fcc = Ions.from_ase(bulk('Si', 'fcc', a= 3.405, cubic=True))
    cell = si_fcc.get_cell()
    si_fcc.set_cell(cell * d, scale_atoms=True)
    
    si_bcc = Ions.from_ase(bulk('Si', 'bcc', a= 3.09, cubic=True))
    cell = si_bcc.get_cell()
    si_bcc.set_cell(cell * d, scale_atoms=True)
    
    si_8cd = Ions.from_ase(bulk('Si', 'diamond', a=5.43, cubic=True))
    cell = si_8cd.get_cell()
    si_8cd.set_cell(cell * d, scale_atoms=True)
    
    inputfile = path_file+'/ks_calculations/Si_dhcp/DATA2/Si_dhcp'+str(i)+'.in'
    si_dhcp = Ions.from_ase(ase.io.read(inputfile))

    inputfile = '/Users/Valeria/Documents/aiWT/Model_ML_function/training_set_data/ks_calculations/Transition_Btin_HD/0.5/si_0.5_'+str(i)+'_4_Btin_hd.in'
    si_Btin_hd = Ions.from_ase(ase.io.read(inputfile))

    inputfile    = '/Users/Valeria/Documents/aiWT/Model_ML_function/training_set_data/ks_calculations/Si_bct5/DATA2/Si'+str(i)+'.in'
    si_bct5 = Ions.from_ase(ase.io.read(inputfile))
    
    inputfile     = '/Users/Valeria/Documents/aiWT/Model_ML_function/training_set_data/ks_calculations/Si_hcp/DATA2/Si_hcp'+str(i)+'.in'
    si_hcp = Ions.from_ase(ase.io.read(inputfile))
    
    inputfile      = '/Users/Valeria/Documents/aiWT/Model_ML_function/training_set_data/ks_calculations/Si_sh/DATA2/Si_sh'+str(i)+'.in'
    si_sh = Ions.from_ase(ase.io.read(inputfile))
    
    traj_Btin.append(si_Btin)
    tral_fcc.append(si_fcc)
    traj_bcc.append(si_bcc)
    traj_8cd.append(si_8cd)
    phases = [si_Btin, si_fcc, si_bcc, si_8cd, si_dhcp, si_Btin_hd, si_bct5, si_hcp, si_sh] 

    sm = SineMatrix(n_atoms_max=65, sparse=False)
    sine_matrices = sm.create(phases, n_jobs=2)
    sine_gradient = np.gradient(sine_matrices)
    Sine_matrices.append(sine_matrices)
    Sine_gradient.append(sine_gradient)
    esm = EwaldSumMatrix(n_atoms_max=8)
    ewal_matrices = esm.create(phases, n_jobs=2)
    Ewal_matrices.append(ewal_matrices)

Python(35857) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(35858) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [5]:
import ase
traj_rand4 = []
traj_rand8 = []
Sine_matrices = []
Sine_gradient = []
for i in np.arange(120, 200, 10):
    if i==0:
        i=1
    if i==160 or i==170 or i==180:
        continue
    ions_4 = ase.io.read('/Users/valeria/Documents/aiWT/Model_ML_function/training_set_data/ks_calculations/random_training/DATA2/si_'+str(i)+'_4.in')
    ions_8 = ase.io.read('/Users/valeria/Documents/aiWT/Model_ML_function/training_set_data/ks_calculations/random_training/DATA2/si_'+str(i)+'_8.in')
    phases = [ions_4, ions_8]
    sm = SineMatrix(n_atoms_max=65, sparse=False)
    sine_matrices = sm.create(phases, n_jobs=2)
    sine_gradient = np.gradient(sine_matrices)
    Sine_matrices.append(sine_matrices)
    Sine_gradient.append(sine_gradient)

In [13]:
traj = traj_Btin + tral_fcc + traj_bcc + traj_8cd + traj_li

In [4]:
soap = SOAP(
        species=["Si"],
        periodic=False,
        rcut=10.0,
        sigma=0.5,
        nmax=10,
        lmax=0,
    )
derivative, soap_matrix = soap.derivatives(
            traj
            )

#### Save the data

In [47]:
np.asarray(Sine_matrices).shape
np.save('SM_Btin_fcc_bcc_8cd_dhcp_Btin-hd_bct5_hcp_sh_65.npy', np.asarray(Sine_matrices))
np.save('SG_Btin_fcc_bcc_8cd_dhcp_Btin-hd_bct5_hcp_sh_65.npy', np.asarray(Sine_gradient))

In [2]:
import numpy as np
Sine = np.load('SM_Btin_fcc_bcc_8cd_dhcp_Btin-hd_bct5_hcp_sh_65.npy')
Sine.shape

(30, 6, 4225)

In [5]:
np.save('Sine_matrices_big_dhcp_phases.npy', np.asarray(Sine_matrices))
np.save('Sine_gradient_big_dhcp_phases.npy', np.asarray(Sine_gradient))

In [11]:
np.save('Ewal_matrices.npy', np.asarray(Ewal_matrices))
# np.save('Soap_matrix.npy', soap_matrix, allow_pickle=True)

In [14]:
np.save('traj_Btim.npy', traj_Btin)
np.save('traj_fcc.npy', tral_fcc)
np.save('traj_bcc.npy', traj_bcc)
np.save('traj_8cd.npy', traj_8cd)
np.save('traj_7cd.npy', traj_7cd)
np.save('traj_li.npy', traj_7cd)
np.save('traj.npy', traj)

/home/valeria/anaconda3/lib/python3.9/site-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)
